# Plot Q vs P
This scrip is meant to create Q vs. P plots for all catchments. 

In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm

from camelsp import Bundesland, util


/home/florian/local/anaconda3/envs/camelsp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with Bundesland('DEC') as bl:
    dec_meta = bl.metadata

dec_meta.head()

,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,area,x,y,q_count,w_count
1372,DEC10000,1271120,./DEC/DEC10000/DEC10000_data.csv,DEC,Saarland,"44,200km²",4.095083e+06,2.947311e+06,22707.0,22707.0
1373,DEC10010,1122120,./DEC/DEC10010/DEC10010_data.csv,DEC,Saarland,203km²,4.090150e+06,2.907085e+06,21611.0,21611.0
1374,DEC10020,1482120,./DEC/DEC10020/DEC10020_data.csv,DEC,Saarland,4671km²,4.085716e+06,2.914696e+06,0.0,0.0
1375,DEC10030,1251120,./DEC/DEC10030/DEC10030_data.csv,DEC,Saarland,"12,500km²",4.110659e+06,2.943239e+06,6909.0,6909.0
1376,DEC10040,1071120,./DEC/DEC10040/DEC10040_data.csv,DEC,Saarland,"48,40km²",4.102207e+06,2.948140e+06,0.0,0.0


Create the correlation coefficients, so we can later select which catchments to plot. 

In [3]:


for NUTS in util._NUTS_LVL2_NAMES.keys():    
    # process this federal state
    with Bundesland(NUTS) as bl:    
        pearson = []
        spearman = []

        # get the metadata
        meta = bl.metadata

        # load the Data-report for each
        for camels_id in tqdm(meta.camels_id.values):
            p = os.path.join(bl.base_path, 'reports', f'{camels_id}.json')
            
            # check if the report exists
            if not os.path.exists(p):
                pearson.append(None)
                spearman.append(None)
                continue
            
            with open(p, 'r') as f:
                report = json.load(f)
            
            try:
                q = [o for o in report['correlations']['pearson'] if o['q'] == 1.0][0]
                w = [o for o in report['correlations']['pearson'] if o['w'] == 1.0][0]
                pearson.append(q['w'])
            except:
                pearson.append(None)
            
            try:
                q = [o for o in report['correlations']['spearman'] if o['q'] == 1.0][0]
                w = [o for o in report['correlations']['spearman'] if o['w'] == 1.0][0]
                spearman.append(q['w'])
            except:
                spearman.append(None)
        
        # all collected, return now
        corrs = pd.DataFrame({'camels_id': meta.camels_id.values, 'q_w_pearson': pearson, 'q_w_spearman': spearman})

        # update
        bl.update_metadata(corrs)

metadata = util.get_metadata()
metadata



100%|██████████| 540/540 [00:00<00:00, 80628.11it/s]
0it [00:00, ?it/s]
100%|██████████| 382/382 [00:00<00:00, 181206.08it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 437/437 [00:00<00:00, 195573.07it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:00<00:00, 94270.84it/s]


,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,area,x,y,q_count,w_count,q_w_pearson,q_w_spearman
0,DEG10000,57300.0,./DEG/DEG10000/DEG10000_data.csv,DEG,Thüringen,NaN,NaN,NaN,29646.0,29646.0,NaN,NaN
1,DEG10010,44700.0,./DEG/DEG10010/DEG10010_data.csv,DEG,Thüringen,NaN,NaN,NaN,22707.0,22707.0,NaN,NaN
2,DEG10020,57420.0,./DEG/DEG10020/DEG10020_data.csv,DEG,Thüringen,NaN,NaN,NaN,35490.0,35490.0,NaN,NaN
3,DEG10030,57650.0,./DEG/DEG10030/DEG10030_data.csv,DEG,Thüringen,NaN,NaN,NaN,12845.0,12845.0,NaN,NaN
4,DEG10040,57021.0,./DEG/DEG10040/DEG10040_data.csv,DEG,Thüringen,NaN,NaN,NaN,21246.0,21246.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3655,DE215350,56113404,./DE2/DE215350/DE215350_data.csv,DE2,Bayern,8.2,4.456659e+06,3009715.476,16497.0,16497.0,NaN,NaN
3656,DE215360,56114000,./DE2/DE215360/DE215360_data.csv,DE2,Bayern,14.1,4.457008e+06,3011664.641,20880.0,20880.0,NaN,NaN
3657,DE215370,56122008,./DE2/DE215370/DE215370_data.csv,DE2,Bayern,84.3,4.465052e+06,3016792.655,23072.0,23072.0,NaN,NaN
3658,DE215380,56143008,./DE2/DE215380/DE215380_data.csv,DE2,Bayern,92.4,4.462157e+06,3021202.262,23440.0,23440.0,NaN,NaN
